<h4>COMP90024 Project Team 1 <br>
Authors:<br>
- Henrik Hao (1255309)<br>
- Haoyi Li (1237964)<br>
- Zilin Su (1155122)<br>
- Angela Yifei Yuan (1269549) </h4>

In [ ]:
# install and import
# pip install seaborn
# pip install folium

import requests
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.wkt import loads
import folium
import warnings

from branca.element import Template, MacroElement
warnings.filterwarnings('ignore', category=FutureWarning) 

## Two scenarios:
### Do the bushfires affect the NSW air quality? 🔥🔥🔥


### Are respiratory diseases caused by poor air quality? 😷😷😷






## Motivation
* #### Since the year 2000, bushfires in the peri-urban areas of southeastern Australia have resulted in the loss of more than 200 lives and necessitated emergency assistance for nearly 18000 individuals.
* #### The long-term effects of bushfires on the air quality should not be ignored.
* #### Related to our life and people are active to talk about weather and air quality on Mastodon

<img src="pictures_in_markdown\mastodon-word.png" width="700" height="300">

<img src="pictures_in_markdown/weather_topic_frequency.jpg" width="650" height="400">




In [ ]:
# Use api to get bushfire
bushfire = (requests.get('http://localhost:9090/bushfireget')).json()
bushfire_df = pd.DataFrame(bushfire)


In [ ]:
def plot_annual_bushfire_trends(bushfire_df):
    '''Define a function to plot annual bushfire trends
    Parameters:
    - bushfire_df: A dataframe about bushfire
    Returns:
        None, but will show plots
    '''
    # Ensure ignition_date is in datetime format and set as index
    if not pd.api.types.is_datetime64_any_dtype(bushfire_df['ignition_date']):
        bushfire_df['ignition_date'] = pd.to_datetime(bushfire_df['ignition_date'])
    if bushfire_df.index.name != 'ignition_date':
        bushfire_df = bushfire_df.set_index('ignition_date')
        
    # Calculate annual total fire area and number of fires
    annual_fire_area = bushfire_df['area_ha'].resample('AS').sum()
    annual_fires = bushfire_df.resample('AS').size()

    # Create subplots
    fig, axs = plt.subplots(2, 1, figsize=(8, 8))
    
    # Plot for total fire area
    axs[0].plot(annual_fire_area, label='Total Fire Area', color='red')
    axs[0].set_title('Annually Total Fire Area in NSW')
    axs[0].set_ylabel('Total Fire Area (hectares)')
    axs[0].grid(True)

    # Plot for number of fires
    axs[1].plot(annual_fires, label='Number of Fires', color='blue')
    axs[1].set_title('Annually Fire Occurrences in NSW')
    axs[1].set_xlabel('Year')
    axs[1].set_ylabel('Number of Fires')
    axs[1].grid(True)

    # Show the plots
    plt.tight_layout()
    plt.show()
    return




### <span style="color:Yellow">**Relationship between bushfires and air quality**</span>

#### What we found:
* #### 2019 was the most severe year for bushfires, with the highest occurrences and largest total areas.
* #### After 2019, the number and area of fires decreased significantly (maybe due to COVID-19?).

In [ ]:
plot_annual_bushfire_trends(bushfire_df)

In [ ]:
def plot_pivot_table_annually_monthly(bushfire_df):
    '''Define a function to plot fire occurences per year per month
    Parameters:
    - bushfire_df: A dataframe about bushfire
    Returns:
        None, but will show plots
    '''
    # extract year and month
    if not pd.api.types.is_datetime64_any_dtype(bushfire_df['ignition_date']):
        bushfire_df['ignition_date'] = pd.to_datetime(bushfire_df['ignition_date'])
    if bushfire_df.index.name != 'ignition_date':
        bushfire_df = bushfire_df.set_index('ignition_date')

    bushfire_df['year'] = bushfire_df.index.year
    bushfire_df['month'] = bushfire_df.index.month

    # frequency of bushfire per month
    fire_counts = bushfire_df.pivot_table(index='year', columns='month', aggfunc='size', fill_value=0)

    plt.figure(figsize=(8, 6))
    sns.heatmap(fire_counts, annot=True, fmt="d", cmap='YlOrRd', linewidths=.5)
    plt.title('Fire Occurrences by Month and Year')
    plt.xlabel('Month')
    plt.ylabel('Year')
    plt.show()

    return


#### What we found:
* #### Summer is the season with the highest incidence of fires (from September to December and January)
* #### Few occurrences of bushfires in winter (from April to July).
* #### Notably, November, December of 2019, and October of 2013 are the worst bushfire periods.

In [ ]:
plot_pivot_table_annually_monthly(bushfire_df)

#### However, 
#### the map below shows that although the frequency of fires was high in 2013, they were small and spread out, whereas there were more large fires in 2019, and they were more concentrated.

<img src="pictures_in_markdown/2016%20and%202019%20fire.png" width="800" height="350">

In [ ]:
# get the average air quality each month (PM2.5, PM10 and Ozone) from 2013 to 2021 
air_quality_year = (requests.get('http://localhost:9090//combine-bushfire/start/2013/end/2021')).json() 
air_quality_year = pd.DataFrame(air_quality_year)
air_quality_year

In [ ]:
def air_quality_measure_annually_monthly(air_quality_year):
    pivot_pm25 = air_quality_year.pivot_table(index='Year', columns='Month', values='Average PM2.5', aggfunc='mean')
    pivot_pm10 = air_quality_year.pivot_table(index='Year', columns='Month', values='Average PM10', aggfunc='mean')
    # pivot_ozone = air_quality_year.pivot_table(index='Year', columns='Month', values='Average Ozone', aggfunc='mean')

    sns.set(style="white")
    # PM2.5
    plt.figure(figsize=(8, 6))
    sns.heatmap(pivot_pm25, annot=True, cmap='YlOrRd', fmt=".1f")
    plt.title('Annual and Monthly Average PM2.5')
    plt.show()

    # PM10
    plt.figure(figsize=(8, 6))
    sns.heatmap(pivot_pm10, annot=True, cmap='YlOrRd', fmt=".1f")
    plt.title('Annual and Monthly Average PM10')
    plt.show()

    # # Ozone
    # plt.figure(figsize=(8, 6))
    # sns.heatmap(pivot_ozone, annot=True, cmap='YlOrRd', fmt=".2f")
    # plt.title('Annual and Monthly Average Ozone')
    # plt.show()
    # return



#### Study shows that ambient ozone concentrations are associated with sinusitis and hay fever, while **particulate matter is associated with more serious respiratory diseases**. Therefore, we will mainly focus on pm2.5 and pm10. 
#### From the pivot tables below, we found:
* #### Air quality in November and December of 2019 was inferior, with the highest concentrations of PM2.5 and PM10. 
* #### The result is consistent with results above, so the occurrence of bushfires may affect the air quality.

In [ ]:
air_quality_measure_annually_monthly(air_quality_year)

#### In the first scenario we reckon:
* #### The impact of a large fire should be significant and affect the entire NSW, 
* #### while, impact of small fire should be insignificant and only affect the surrounding air quality. 


#### The data was selected from 2016 to 2021. 2019 was the worst year for fires, and 2016 was a relatively normal year for fires in the selected range (not considering the year attacked by COVID-19). Therefore, we will focus on the data of 2016 and 2019.

In [ ]:
def find_site_each_year(year):
    columns = ['siteid', 'phacode', 'timestamp', 'geometry', 'sitename', 'pm10', 'pm2p5', 'ozone']
    response = requests.get(f'http://localhost:9090/getairquality/yearsites/{year}')
    site_year = response.json()

    site_dfs = [pd.DataFrame([site['_source'].values()], columns=columns) for site in site_year]

    site_year_clean = pd.concat(site_dfs, ignore_index=True).dropna(subset=['pm10', 'pm2p5', 'ozone'], how='all')
    
    site_year_clean['timestamp'] = pd.to_datetime(site_year_clean['timestamp'])
    site_year_clean = site_year_clean.set_index('timestamp')
    site_year_clean['year'] = site_year_clean.index.year
    site_year_clean['month'] = site_year_clean.index.month

    site_year_clean['geometry'] = site_year_clean['geometry'].apply(loads)
    site_year_clean = gpd.GeoDataFrame(site_year_clean, geometry='geometry')
    site_year_clean = site_year_clean.set_crs("EPSG:4326")

    return site_year_clean

In [ ]:
def find_near_far_site(year, site_year_df, index_name, bushfire_df):
    bushfire_df['ignition_date'] = pd.to_datetime(bushfire_df['ignition_date'])
    bushfire_df = bushfire_df.set_index('ignition_date')
    bushfire_df['year'] = bushfire_df.index.year
    bushfire_df['month'] = bushfire_df.index.month
    bushfire_df['geometry'] = bushfire_df['geometry'].apply(loads)
    bushfire_gdf = gpd.GeoDataFrame(bushfire_df, geometry='geometry')
    bushfire_gdf = bushfire_gdf.set_crs("EPSG:4326")
    
    bushfire_year_gdf = bushfire_gdf[bushfire_gdf['year'] == year]
    merged_gdf = pd.merge(site_year_df, bushfire_year_gdf, on='month')

    points = merged_gdf[['month', 'siteid', 'geometry_x', index_name]].copy().drop_duplicates()
    polygons = merged_gdf[['month', 'geometry_y', 'area_ha']].copy().drop_duplicates()

    nearest_farthest_data = []

    # Only compare distance under the same month 
    for month in polygons['month'].unique():
        month_polygons = polygons[polygons['month'] == month]
        month_points = points[points['month'] == month]
        
        for index, poly in month_polygons.iterrows():
            # Calculate distance between site and fire polygon
            month_points['distance'] = month_points.apply(lambda row: row['geometry_x'].distance(poly['geometry_y']), axis=1)
            # Find the nearest and farthest site
            nearest_point = month_points.loc[month_points['distance'].idxmin()]
            farthest_point = month_points.loc[month_points['distance'].idxmax()]

            nearest_farthest_data.append({
                'month': month,
                'area': poly['area_ha'],
                'Nearest Site ID': nearest_point['siteid'],
                f'{index_name}_near': nearest_point[index_name],
                f'{index_name}_far': farthest_point[index_name],
                'Farthest Site ID': farthest_point['siteid'],
            })

    nearest_farthest_df = pd.concat([pd.DataFrame([data]) for data in nearest_farthest_data], ignore_index=True)
    
    return nearest_farthest_df

#### Observing the concentration of PM2.5 and PM10 at the **nearest** and **farthest** sites is an intuitive way.

In [ ]:
def bushfire_site_box_plot(year, index, site_year_df, bushfire_df):
    ### large range bushfire
    near_far_year = find_near_far_site(year, site_year_df, index, bushfire_df)
    large_bushfire = near_far_year[near_far_year['area'] > 100]
    melted_large = large_bushfire.melt(id_vars=['month', 'area', 'Nearest Site ID'], 
                        value_vars=[f'{index}_near', f'{index}_far'], 
                        var_name='Measurement', value_name='Value')

    ### small range bushfire

    small_bushfire = near_far_year[near_far_year['area'] <= 5]
    melted_small = small_bushfire.melt(id_vars=['month', 'area', 'Nearest Site ID'], 
                        value_vars=[f'{index}_near', f'{index}_far'], 
                        var_name='Measurement', value_name='Value')


    plt.figure(figsize=(6, 4))
    sns.boxplot(x='month', y='Value', hue='Measurement', data=melted_small)
    plt.title(f'Monthly Comparison of {index}_near and {index}_far for small bushfire in {year}')
    plt.xlabel('Month')
    plt.ylabel('Concentration')
    plt.legend(title='Measurement')
    plt.grid(True)
    plt.show()


    plt.figure(figsize=(6, 4))
    sns.boxplot(x='month', y='Value', hue='Measurement', data=melted_large)
    plt.title(f'Monthly Comparison of {index}_near and {index}_far for large bushfire in {year}')
    plt.xlabel('Month')
    plt.ylabel('Concentration')
    plt.legend(title='Measurement')
    plt.grid(True)
    plt.show()
    return 

### For 2016

#### The box plots below represent:

* #### There were more small fires than large fires in 2016. 
* #### In Winter, when there are fewer fires, the PM10 concentration measured at the nearest site is even lower than that at the farthest site in other months.

In [ ]:
bushfire_site_box_plot(2016, 'pm10',find_site_each_year(2016), bushfire_df)


### For 2019

#### The overall air quality is much worse than that in 2016. 
* #### Because there were many large fires in 2019. 
#### The difference between the nearest and farthest concentration is slight for large fires. 
* #### As we envisioned, the impact of large fires on the entire NSW cannot be underestimated.

In [ ]:
bushfire_site_box_plot(2019, 'pm10',find_site_each_year(2019), bushfire_df)


### <span style="color:Yellow">**Relationship between admission and bushfires**</span>

### What we found:

* #### Compared with 2016-2019, the number of respiratory admissions in 2020 and 2021 shows a significant decrease. 

* #### People are more likely to get respiratory diseases in winter.

<img src="pictures_in_markdown/2016_2021_admissions_nsw.png" width="800" height="500">







<br>

#### To show the relationship between the number of admissions and the frequency of bushfires, we analyzed the data in two aspects:

- #### Compared the percentage of admissions and the percentage of bushfires each year from 2016 to 2021.
- #### Compared the percentage of admissions and the percentage of bushfires each month from 2016 to 2021.

#### We chose to use percentages to scale the number of bushfires and the number of admissions to the same numeric level.

<img src="pictures_in_markdown/admision_bushfire_yealy.png" width="600" height="400">
<img src="pictures_in_markdown/admission_bushfire_monthly.png" width="600" height="400">

#### The left graph shows that when fires are widespread, the increase in admission rates is not very significant. 

#### The right illustrates that bushfires usually occur in summer, while the number of admissions typically increases in winter. <br> This indicates that the admission rate may be more related to the seasons (temperatures).

<br>
<br>

<img src="pictures_in_markdown/admission_bushfire_airquality.png" width="600" height="400">



### In conclusion, bushfires affect air quality and that large bushfires have a much more comprehensive range of impacts that can cover the entire NSW. However, although air quality affects the respiratory system, the development of respiratory disease is not solely determined by air quality.

